# Proyecto Grupo 36 - Comedor Social

Esta es la implementación en Gurobi del proyecto presentado por el grupo 36.

Dado N días, consideramos los siguientes parámetros:

* $g_i$: cantidad de gente que llega el día $i$.
* $c_a$: costo del alimento $a$.
* $cs$: sueldo fijo del trabajor remunerado. (mensual)
* $dm_i$: donaciones monetarias recibidas el día $i$ (total, de todas las fuentes).
* $dc_{ia}$: cantidad de alimento $a$ que fue recibido como donación el día $i$.
* $e$: entrada que se cobra para comer (200 CLP)
* $d_a$: duración del alimento $a$
* $ap_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es rico en proteinas.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $ac_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es rico en carbohidratos.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $av_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es una verdura.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $af_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es una fruta.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $v_a$: volumen que ocupa el alimento $a$.
* $vmax$: volumen máximo de alimentos que se pueden almacenar en despensa.
* $amax_i$: cantidad máxima de alimentos que los cocineros están dispuestos a incluir en la confección de un plato el día $i$.

Estos parámetros se encuentran en el archivo `data.json`.

In [108]:
from gurobipy import *
import json

from IPython.display import HTML, display
import tabulate

# Create a new model
m = Model("Comedor social")

# parametros

with open('data.json') as f:
    data = json.loads(f.read())

dias = [['Dia', 'Alimentos Máximos', 'Donaciones Diarias', 'Visitantes']]
dias_temp = []
alimentos = [['Alimento', 'Volumen Alimento', 'Duracion Alimento', 'Costo Alimento', 'Verdura', 'Fruta', 'Proteina', 'Carbohidrato']]
alimentos_temp = []

donaciones = [['Alimento'] + [f'Día {n}' for n in range(data['dias'])]]
donaciones_temp = []
    
for k, i in data.items():
    if isinstance(i, (int, float)):
        print(k.capitalize() + ':', i)
    elif isinstance(i, list):
        if k in ('amax', 'donaciones_monetarias', 'visitas'):
            dias_temp.append(i)
        elif k != 'cantidad_alimento':
            alimentos_temp.append(i)
        else:
            donaciones_temp.append(i)

dias.extend(list(map(lambda x: [x[0]] + list(x[1]), ((x, n) for x, n in enumerate(zip(*dias_temp))))))
alimentos.extend(list(map(lambda x: [x[0]] + list(x[1]), ((x, n) for x, n in enumerate(zip(*alimentos_temp))))))
donaciones.extend(list(map(lambda x: [x[0]] + x[1][0], ((x, n) for x, n in enumerate(zip(*donaciones_temp))))))
        
display(HTML(tabulate.tabulate(dias, tablefmt='html')))
display(HTML(tabulate.tabulate(alimentos, tablefmt='html')))
display(HTML(tabulate.tabulate(donaciones, tablefmt='html')))

Dias: 31
Alimentos: 50
Sueldo_fijo: 250000
Vol_max: 500
Entrada: 200


Dia,Alimentos Máximos,Donaciones Diarias,Visitantes
0,5,10520,134
1,4,22485,114
2,4,10925,112
3,5,23287,192
4,6,26257,113
5,7,23275,128
6,5,7323,208
7,5,9477,190
8,6,14557,122
9,5,14149,188


Alimento,Volumen Alimento,Duracion Alimento,Costo Alimento,Verdura,Fruta,Proteina,Carbohidrato
0,1,12,2336,0,1,0,0
1,1,26,100,0,0,1,0
2,1,10,100,0,0,1,1
3,1,21,1801,0,0,1,1
4,1,12,1702,0,0,0,0
5,1,15,1464,0,0,1,0
6,1,10,1193,1,0,0,1
7,1,10,772,0,1,0,0
8,1,10,880,0,1,0,1
9,1,45,474,1,0,1,0


Alimento,Día 0,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,Día 11,Día 12,Día 13,Día 14,Día 15,Día 16,Día 17,Día 18,Día 19,Día 20,Día 21,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30
0,8,6,6,7,8,8,11,4,15,3,5,6,8,0,7,3,0,12,9,3,15,7,5,2,8,5,4,8,13,7,12
1,6,18,4,7,7,4,9,8,4,7,0,7,3,0,13,16,10,5,10,0,4,0,7,4,0,13,0,0,4,10,15
2,7,9,4,0,6,0,19,4,6,10,1,5,6,0,8,13,3,7,2,7,4,5,7,1,6,9,11,12,6,6,9
3,7,8,14,6,0,12,6,10,8,1,15,10,3,10,4,7,12,12,7,9,17,3,10,5,3,14,9,4,10,4,8
4,10,6,12,16,5,4,14,4,0,6,0,13,2,2,11,8,21,7,4,4,1,8,6,12,10,6,17,11,9,6,2
5,9,6,7,9,10,9,3,5,5,12,0,8,9,4,12,1,11,9,10,12,14,0,12,13,5,8,6,5,8,9,12
6,15,13,8,19,8,12,11,12,0,15,10,14,4,8,9,8,0,2,3,13,19,9,13,11,7,3,1,2,8,8,7
7,2,8,8,6,9,4,20,13,9,12,3,1,8,12,6,9,5,17,7,6,8,4,8,13,2,2,4,10,8,6,7
8,13,7,13,6,0,13,6,8,10,12,16,8,7,8,13,0,13,6,7,8,10,11,6,11,6,8,1,12,8,1,7
9,8,9,9,8,10,4,6,6,8,8,0,1,6,23,8,6,10,1,14,18,4,4,10,7,12,0,15,14,4,2,3


## Variables
* $x_{ai}$: cantidad de alimento $a$ que se compra en el día $i$.
* $y_{ai}$ : cantidad de alimento $a$ que se utiliza en el día $i$
* $I_{ai}$: cantidad del alimento $a$ que se almacena el día $i$.

In [109]:
# Create variables
X = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} comprado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]
Y = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} usado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]
I = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} almacenado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]

## Función objetivo
$$\text{min} \sum_{a \in A} \sum_{i \in D} c_a x_{ai}$$

In [75]:
# Set objective
m.setObjective(sum(sum(data['costo_alimento'][a] * X[i][a] for a in range(data['alimentos'])) for i in range(data['dias'])), GRB.MINIMIZE)

# Restricciones

* Límite de gastos: el gasto diario de dinero no puede superar lo obtenido en donaciones ese día más lo que haya sobrado (lo obtenido menos lo gastado) de los días anteriores. Se incluye el costo diario que significa tener un trabajador remunerado.
    $$\sum_{a \in A} c_a x_{ai} \leq dm_i + \sum_{k=1}^{i-1} (dm_k + g_k e - \frac{cs}{30.5} - \sum_{a \in A} c_a x_{ak})   \quad\quad\quad     \forall i \in D$$
* No se puede usar más de lo que se tiene
    $$y_{ai} \leq x_{ai} + I_{a(i-1)}     \quad\quad\quad     \forall a \in A, \forall i \in D$$
* Inventario inicial
    $$I_{a0} = 0     \quad\quad\quad     \forall a \in A$$
* Límite de almacenamiento
    $$\sum_{a \in A} I_{ai} v_a \leq vmax     \quad\quad\quad     \forall i \in D$$
* Una proteina mínimo por comida
    $$ \sum_{a \in A} y_{ai}  ap_a \geq g_i \quad\quad\quad \forall i \in D $$
* Un carbohidrato mínimo por comida
    $$ \sum_{a \in A} y_{ai}  ac_a \geq g_i \quad\quad\quad \forall i \in D $$
* Una verdura mínimo por comida
    $$ \sum_{a \in A} y_{ai}  av_a \geq g_i \quad\quad\quad \forall i \in D $$
* Una fruta mínimo por comida
    $$ \sum_{a \in A} y_{ai}  af_a \geq g_i \quad\quad\quad \forall i \in D $$
* Máximo de alimentos por plato de comida
    $$ \sum_{a \in A} (y_{ai} af_a + y_{ai} av_a + y_{ai} ac_a + y_{ai} ap_a) \leq amax_i g_i \quad\quad\quad \forall i \in D$$
* Caducidad de los Alimentos: para cada día y cada alimento, la totalidad de alimentos de cierto tipo recibidos (ya sea por compra o por donación) debe haberse utilizado en tantos días en el futuro como la caducidad de ese alimento. Como los alimentos más viejos siempre se utilizan primero, esto significa que para la fecha que los alimentos vencerian estos ya deben haber sido utilizados. La condición también se cumple en días que no ha habido compra o donación, siempre y cuando se cumpla en el día de la última compra/donación.
    $$ \sum_{j=1}^{i} (dc_{ja} + x_{aj}) \leq \sum_{j=1}^{i + d_a} y_{ja} \quad\quad\quad \forall a \in A, \forall i \in D$$
* Naturaleza de variables
    $$x_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$y_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$I_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$x_{ai}, y_{ai}, I_{ai} \in \mathbb{N}\cup\{0\}     \quad\quad\quad     \forall a \in A, \forall i \in D$$

In [76]:
# Add constraint
for a in range(data['alimentos']):
    m.addConstr(I[0][a] <= 0, f"Inventario inicial del alimento {a}")

for i in range(data['dias']):
    m.addConstr(sum(data['costo_alimento'][a] * X[i][a] for a in range(data['alimentos'])) <= data['donaciones_monetarias'][i] + data['visitas'][i] * data['entrada'] - (data['sueldo_fijo'] / 30.5) + sum(data['donaciones_monetarias'][k] + data['visitas'][k] * data['entrada'] - (data['sueldo_fijo'] / 30.5) - sum(data['costo_alimento'][c] * X[k][c] for c in range(data['alimentos'])) for k in range(0, data['dias'] - 1)),
                f"Límite de gastos en el día {i}")
    m.addConstr(sum(I[i][a] * data['volumen_alimentos'][a] for a in range(data['alimentos'])) <= data['vol_max'],
                f"Límite de almacenamiento día {i}")
    m.addConstr(sum(Y[i][a] * data['proteina'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una proteina mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['carbohidrato'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Un carbohidrato mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['verdura'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una verdura mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['fruta'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una fruta mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * (data['proteina'][a] + data['carbohidrato'][a] + data['verdura'][a] + data['fruta'][a]) for a in range(data['alimentos'])) <= data['amax'][i] * data['visitas'][i],
                f"Máximo de alimentos por plato de comida día {i}")
    for a in range(data['alimentos']):
        m.addConstr(Y[i][a] <= X[i][a] + I[max(0, i - 1)][a],
                f"No se puede usar más de lo que se tiene ")
        m.addConstr(sum(data['cantidad_alimento'][a][j] + X[j][a] for j in range(0, i + 1)) <= sum(Y[k][a] for k in range(0, min(data['dias'] - 1, i + data['duracion_alimentos'][a]))),
                f"Caducidad de los Alimentos")
        m.addConstr(X[i][a] >= 0, f"Naturaleza de X en dia {i} y alimento {a}")
        m.addConstr(Y[i][a] >= 0, f"Naturaleza de Y en dia {i} y alimento {a}")
        m.addConstr(I[i][a] >= 0, f"Naturaleza de I en dia {i} y alimento {a}")
    

# Resultado

In [78]:
m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.X))
    print('%s %g' % (v.varName, v.Y))
    print('%s %g' % (v.varName, v.I))

print('Obj: %g' % m.objVal)

Optimize a model with 8017 rows, 4650 columns and 126365 nonzeros
Variable types: 0 continuous, 4650 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolved: 2379 rows, 4379 columns, 75033 nonzeros

Continuing optimization...


Explored 0 nodes (2653 simplex iterations) in 0.11 seconds
Thread count was 2 (of 2 available processors)

Solution count 1: 49600 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.960000000000e+04, best bound 4.960000000000e+04, gap 0.0000%
Alimento 0 comprado dia 0 -0


AttributeError: 'gurobipy.Var' object has no attribute 'Y'